<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/Products_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import

In [ ]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv
import numpy as np

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#Go to folder

Hay que considerar que se trabajara con las bases de datos (cualquier formato) y con las carpetas de control de product. Por ello, es que se inicia en la carpeta que permite seleccionar esto. Recordando:
*  Masters: Se encuentran las bases de datos, organizadas por tipo de archivo.
*  Origin: Primeros experimentos, se encuentran los archivos originales por consulta, divididos por país y categoría del país.
*   Products: Dónde se pretende almacenar las base de características de productos.

In [ ]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
Masters  Origin  Products


#Tools

##Centered Len

#Count if

In [ ]:
def count_if(array, value):
    count = 0
    for element in array:
        if element == value:
            count += 1
    return count

#Standarize dates

In [ ]:
def standarize_dates(date):
    check = False
    try:
        date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S.%f')
        check = True
    except ValueError:
        try:
            date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            check = True
        except:
            try:
                date = datetime.strptime(str(date), '%Y-%m-%d %H_%M')
                check = True
            except:
                pass
    if check == True:
        date = date.strftime('%Y-%m-%d %H:%M')
    
    return date

In [ ]:
df

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
3763,2020-07-31 00:38,14.0,Map Of The Soul: Persona,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Map-Soul-Persona-BTS...,4.8,2464.0,BTS,Audio CD,432.86,None
3773,2020-07-31 00:38,24.0,"Love Yourself: Her (Random cover, incl. 100-pa...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Love-Yourself-Her-BT...,4.7,900.0,BTS,Audio CD,688.00,None
3772,2020-07-31 00:38,23.0,Harry Styles (Edición Especial) (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Harry-Styles-Edici%C...,4.7,1388.0,Harry Styles,Vinil,1223.28,None
3771,2020-07-31 00:38,22.0,Blurryface (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Blurryface-Vinyl-Twe...,4.7,2255.0,Twenty One Pilots,Vinil,561.00,None
3769,2020-07-31 00:38,20.0,Up All Night,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Up-All-Night-One-Dir...,4.7,1108.0,One Direction,Audio CD,127.00,None
...,...,...,...,...,...,...,...,...,...,...,...
19,2020-08-13 18:25,20.0,Is This It (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/This-Vinyl-Strokes/d...,4.3,575.0,The Strokes,Vinil,392.25,None
20,2020-08-13 18:25,21.0,Caifanes,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Caifanes/dp/B00WKNGH...,4.4,132.0,Caifanes,Audio CD,99.00,None
21,2020-08-13 18:25,22.0,Black Pumas (Vinyl),https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Black-Pumas-Vinyl/dp...,NaN,NaN,Black Pumas,Vinil,1163.07,None
10,2020-08-13 18:25,11.0,FINE LINE,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/FINE-LINE-HARRY-STYL...,4.8,781.0,HARRY STYLES,Audio CD,170.00,None


#Lab

##Extract Products

###Name Products

In [ ]:
entry = 'Masters/mx/parquet/mx-master_db_music.parquet'

#Get de Nanmes
df = pd.read_parquet(entry)
df['time'] = df['time'].apply(standarize_dates)
df = df.sort_values(by="time")
product_names_raw = df['Product Names'].tolist()
#Remember, this gets de None Value
products = list(set(product_names_raw))
None_cases = count_if(product_names_raw, None)
None_cases

50

##Scrap Dates

In [ ]:
dates_raw = df['time'].tolist()
dates_time = set(dates_raw)

just_dates = []
for dt in dates_time:
    try:
        date = datetime.strptime(dt,'%Y-%m-%d %H:%M').date()
    except:
        date = datetime.strptime(dt,'%Y-%m-%d %H_%M').date()

    if not date in just_dates:
        just_dates.append(date)

just_dates.sort()
print(just_dates)

[datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13)]


#Obtaining All Ranks

In [ ]:
products_dict = {products[n]:n for n in range(len(products))}
# products_dict

In [ ]:
ranks = df['Rank'].tolist()
day_arrays = [None] * len(just_dates)
rank_day_array = [None] * len(products)

d_counter = 0
first_date = just_dates[0]

for r in range(len(ranks)):
    
    #Day that we are
    try:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H:%M').date()
    except:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H_%M').date()

    if date_check != first_date:
        day_arrays[d_counter] = rank_day_array
        rank_day_array = [None] * len(products)
        if d_counter < (len(just_dates)-1):
            d_counter += 1
            first_date = just_dates[d_counter]

    if date_check == first_date:
        # if date_check == just_dates[0]:
        #     print(date_check)
        rank = ranks[r]
        #What product are we talking about
        product_name = product_names_raw[r]
        product_position = products_dict[product_name]
        if rank_day_array[product_position] == None:
            array = []
            rank_day_array[product_position] = array
        else:
            array_rank = rank_day_array[product_position]
            array_rank.append(rank)
        # print(f'{product_name} Position: {product_position} rank: {rank} -> {rank_day_array[product_position]}')

for ranked_day in day_arrays:
    print(ranked_day)

[[39.0, 39.0, 39.0], None, None, None, None, None, None, None, None, [], None, None, None, None, None, None, None, None, None, None, None, None, [], None, None, None, None, [6.0, 6.0, 6.0, 6.0, 11.0, 11.0, 11.0, 11.0, 11.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0], None, None, None, [12.0, 12.0, 12.0, 12.0, 12.0], None, None, None, None, None, [25.0, 25.0, 25.0, 25.0], None, None, None, None, None, None, None, None, None, None, [9.0, 9.0, 9.0, 6.0, 6.0, 6.0, 6.0, 6.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [], None, None, None, [], None, None, None, [24.0, 24.0, 24.0, 24.0, 37.0, 37.0, 37.0, 37.0, 37.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0], None, None, None, [38.0, 38.0, 38.0, 38.0, 38.0], None, None, [12.0, 12.0, 12.0, 12.0, 18.0, 18.0, 18.0, 18.0, 18.0, 34.0, 34.0, 34.0, 34.0, 34.0, 34.0], [46.0, 46.0, 46.0, 46.0], None, None, None, [47.0, 47.0, 47.0], None, None, None, None, [49.0, 49.0, 49.0], None, None, None, None, None, None, None, [14.0, 14.0, 14.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.